In [94]:
library('dplyr')
library("zoo")
directory = '/media/nicolas/Elements/data/'
time_past = 1970
time_pre = 2010
# STAID='000013'
# time = time_pre

In [95]:
openfile <- function(path_to_data,time){
    
        # Vérification de l'éxistence du fichier    
    if (!file.exists(path_to_data)){
            stop("Invalid ID")
    }
    
    Data <- read.table(path_to_data,skip = 20,header=TRUE,sep=',')
    Data <- filter(Data,Data[,5]==0)               
    Data <- Data[,-c(1,2,5)]
    Data$DATE <- strptime(Data$DATE, format = "%Y%m%d")
 
    Data <- filter ( Data, Data$DATE$year > time - 1905 & Data$DATE$year <= time - 1900)
    if (!length(Data$DATE$year)==1826){
        stop('Unvailable data for this period')
    }
    return(Data)
}

In [96]:
Tempannualrange <- function(STAID,time){
    path_to_data_min <- paste(directory,'TN/TN_STAID',STAID,'.txt',sep='')
    path_to_data_max <- paste(directory,'TX/TX_STAID',STAID,'.txt',sep='')
    

    if (!time == time_pre) {
        if(!time == time_past){
        stop('Time argument must be time_pre or time_past')
    }}
 
    Data_min <- openfile(path_to_data_min,time)
    Data_max <- openfile(path_to_data_max,time)
    
    #On extrait de la colonne de date un vecteur ne contenant que les années
    vec_year<-as.numeric(format(Data_max$DATE,"%Y"))
    #On extrait de la colonne de date un vecteur ne contenant que les mois
    vec_month<-as.numeric(format(Data_max$DATE, "%m"))

    #Calcule de la température min de tout les mois de toutes les année : 
    data_annual_monthly_min<-aggregate(Data_min$TN,by = list(vec_year,vec_month),min)

    #Calcule de la température max de tout les mois de toutes les année : 
    data_annual_monthly_max<-aggregate(Data_max$TX,by = list(vec_year,vec_month),max)
    
    data_monthly_min <- aggregate(data_annual_monthly_min$x,data_annual_monthly_min[2],mean)
    data_monthly_max <- aggregate(data_annual_monthly_max$x,data_annual_monthly_max[2],mean)
    data_monthly_range <- data_monthly_min
    data_monthly_range$max <- data_monthly_max$x
    data_monthly_range<-mutate(data_monthly_range,range = data_monthly_range$max-data_monthly_range$x)
    BIO2 <- mean(data_monthly_range$range)
    BIO5 <- max(data_monthly_max$x)
    BIO6 <- min(data_monthly_min$x)
    BIO7 <- BIO5-BIO6
    BIO3 <- (BIO2/BIO7)*100
    
    return(list(BIO2,BIO3,BIO5,BIO6,BIO7))
}

In [97]:
k<-Tempannualrange('000013',time=time_past)
k
h<-Tempannualrange('000013',time=time_pre)
h

[[1]]
[1] 249.1

[[2]]
[1] 50

[[3]]
[1] 320.2

[[4]]
[1] -178

[[5]]
[1] 498.2

[[1]]
[1] 248.3667

[[2]]
[1] 54.68223

[[3]]
[1] 331.2

[[4]]
[1] -123

[[5]]
[1] 454.2